In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tools import CN
import matplotlib.pylab as plt
from diffrl.env import DifferentiablePhysicsEnv

# 1.19

env = DifferentiablePhysicsEnv.parse(
    #cfg_path='../diffrl/env/configs/plb_cuda.yml',
    #sim_cfg=dict(yield_stress=1000.),

    cfg_path='configs/plb3d.yml',
    sim_cfg=dict(quality=1.0, yield_stress=1000.),
    use_taichi=True,

    task_cfg=dict(TYPE="MoveObject", shape_metric=dict(chamfer=1., center=10.)),
    solver=dict(
        render_path=None,
        render_interval=50,
        max_iter=1, lr=0.1
    ),
    **{"manipulator_cfg.size": (0.05, 0.2, 0.2)},
)

Session:
  signature_scheme="hmac-sha256": --Session.key=b"3ca52039-1550-4a28-afbf-c26e7c6f564f"
control=9001: --hb=9000
iopub=9004: --f=/tmp/tmp-1326738Zn3zJsn6w0.json
ip=127:
  0:
    0:
      1: --stdin=9003
shell=9002: --transport="tcp" 
 ------------------------------------------------------------ 
 cfg_path: configs/plb3d.yml
device: cuda:0
manipulator_cfg:
  CLASS: Manipulator
  TYPE: ParallelGripper
  size: (0.05, 0.2, 0.2)
max_episode_steps: 50
observer_cfg:
  CLASS: Observer
  TYPE: TaichiRGBDObserver
sim_cfg:
  quality: 1.0
  yield_stress: 1000.0
solver:
  compute_loss_in_end: False
  early_stop: None
  lr: 0.1
  max_iter: 1
  optim_type: Adam
  render_interval: 50
  render_path: None
  save_state: 1
  verbose: True
task_cfg:
  CLASS: Task
  TYPE: MoveObject
  shape_metric:
    center: 10.0
    chamfer: 1.0
use_taichi: True
warning_off: True
ERROR while adding argumetns before!!! 'Non-existent config key: control=9001'
E: 5000.0
dim: 3
dt_quality: 1.0
dtype: float32
dynamic_

In [3]:
# out = env.solve(env.task.reset(env), [env.manipulator.get_initial_action() for i in range(50)], max_iter=30)

In [114]:
import torch
import pickle

with open('test_data.pkl', 'rb') as f:
    out = pickle.load(f)

actions = torch.nn.Parameter(torch.tensor(out['best_action'], dtype=torch.float32, device='cuda:0'))
env.set_state(out['initial_state'])

T = 50

sim = env.simulator
sim.substeps = 2
func = env.torch_env

func.zero_grad()
obs = func.get_obs(0, device='cuda:0')
loss = 0
for idx, a in enumerate(actions[:T]):
    obs = func.forward(idx, a, *obs)
    #loss = loss + obs[0][:, -2:].mean() + ((obs[0][:, :3] - torch.tensor([0.2, 0.5, 0.4], device='cuda:0'))**2).mean()
    if idx == T-1:
        loss = loss + ((obs[0][:, :3] - torch.tensor([0.2, 0.5, 0.4], device='cuda:0'))**2).mean()
    # print(((obs[0][:, :3] - torch.tensor([0.2, 0.5, 0.4], device='cuda:0'))**2).mean())


print(loss)

loss.backward()
print(actions.shape)
print(actions.grad[:T])

tensor(0.1127, device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([50, 12])
tensor([[ 6.2510e-04, -8.0480e-05,  3.8276e-05,  4.8309e-07,  2.6212e-05,
         -1.6181e-04,  4.3763e-03,  1.4586e-04, -1.5115e-04,  3.1221e-07,
         -6.4204e-06,  2.7131e-04],
        [ 6.0702e-04, -9.5618e-05,  4.9681e-05,  1.6213e-06,  2.4641e-05,
         -1.6266e-04,  4.4456e-03,  1.1374e-04, -1.3092e-04,  1.2321e-06,
         -4.5019e-06,  2.7171e-04],
        [ 6.7484e-04, -3.6650e-04,  2.0326e-04,  6.6252e-06,  1.5869e-05,
         -1.8047e-04,  4.4658e-03, -3.9460e-05, -3.8873e-05,  2.6847e-06,
          1.0731e-06,  2.8017e-04],
        [ 6.9546e-04, -2.1389e-04,  1.0886e-04,  4.7515e-06,  2.4904e-05,
         -1.6553e-04,  4.4900e-03, -4.3246e-05, -4.2336e-05,  2.3435e-06,
         -3.6759e-07,  2.7902e-04],
        [ 6.9030e-04, -1.0463e-03,  5.5756e-04,  1.4489e-05, -3.6189e-06,
         -2.1849e-04,  4.5263e-03,  9.0684e-06, -7.4264e-05,  2.2075e-06,
         -4.5886e-06,  2.7470e-04],
   

In [110]:
# print(env.simulator.primitives[0].position.grad.to_numpy().mean(axis=1)[:80])
print(env.simulator.x.grad.to_numpy()[:, :625].mean(axis=(1,2))[:sim.substeps*T+1])
print(env.simulator.v.grad.to_numpy()[:, :625].mean(axis=(1,2))[:sim.substeps*T+1])


from utils import dump_substeps

out = dump_substeps(sim, 22)
print('dumped')
with open('substep_4.pkl', 'wb') as f:
    pickle.dump(out, f)

[-1.3604297e-04 -6.8023139e-05 -6.8026318e-05 -6.8031033e-05
 -6.8037392e-05 -1.3609085e-04 -6.8055102e-05 -6.8066343e-05
 -6.8079011e-05 -6.8067282e-05 -6.8126195e-05 -6.8322399e-05
 -6.8718131e-05 -6.9374481e-05 -7.0327820e-05 -7.1652015e-05
 -7.3445102e-05 -7.5194424e-05 -7.7072720e-05 -7.9124540e-05
 -8.1336329e-05 -8.2300459e-05 -8.3652674e-05 -8.5118969e-05
 -8.6586631e-05 -8.7971486e-05 -8.9285742e-05 -9.0471534e-05
 -9.1465125e-05 -9.2234084e-05 -9.2813912e-05 -9.3259128e-05
 -9.3621144e-05 -9.3908500e-05 -9.4076124e-05 -9.4002142e-05
 -9.3475704e-05 -9.2273956e-05 -9.0736721e-05 -8.9009081e-05
 -8.7197666e-05 -8.6588130e-05 -8.5351843e-05 -8.4115425e-05
 -8.2978047e-05 -8.1873011e-05 -8.1002610e-05 -8.0301077e-05
 -7.9810758e-05 -7.9511927e-05 -7.9503312e-05 -8.0478596e-05
 -8.2172228e-05 -8.3741979e-05 -8.4129613e-05 -8.6311440e-05
 -8.9314875e-05 -9.2133843e-05 -9.6341450e-05 -9.9221244e-05
 -1.0220693e-04 -1.0186648e-04 -1.0222448e-04 -1.0216761e-04
 -1.0139742e-04 -9.98049

In [111]:
if True:
    plb_sim = sim
    with open('substep_4.pkl', 'rb') as f:
        data = pickle.load(f)

    plb_sim.set_state(0, data[0]['state'])
    plb_sim.set_state(1, data[1]['state'])
    plb_sim.clear_grid()
    plb_sim.grid_m.from_numpy(data[2][4].reshape(64, 64, 64))
    plb_sim.grid_v_in.from_numpy(data[2][0].reshape(64, 64, 64, 3))
    plb_sim.grid_op(0)
    plb_sim.grid_v_out.grad.from_numpy(data[2][3].reshape(64, 64, 64, 3))
    for i in plb_sim.primitives.primitives:
        i.clear_grad(10)
    plb_sim.grid_op.grad(0)

    grid_v_out2 = plb_sim.grid_v_out.to_numpy()
    print(grid_v_out2.sum(), data[2][2].sum())
    print((grid_v_out2 - data[2][2]).sum())

    pos_grad = []
    rot_grad = []
    next_pos_grad = []
    next_rot_grad = []
    for i in plb_sim.primitives.primitives:
        pos_grad.append(i.position.grad.to_numpy()[0])
        next_pos_grad.append(i.position.grad.to_numpy()[1])
        rot_grad.append(i.rotation.grad.to_numpy()[0])
        next_rot_grad.append(i.rotation.grad.to_numpy()[1])
    print(pos_grad)
    print(next_pos_grad)
    

    print(rot_grad)
    print(next_rot_grad)
    

235.7989 235.7989
0.0
[array([ 0.02729355,  0.02924888, -0.01842773], dtype=float32), array([0., 0., 0.], dtype=float32)]
[array([-0.02730483, -0.02924368,  0.01841928], dtype=float32), array([0., 0., 0.], dtype=float32)]
[array([-5.9815825e-06, -1.2637790e-03,  2.4523719e-03,  2.3323635e-03],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32)]
[array([ 6.4455280e-06,  1.2637151e-03, -2.4524285e-03, -2.3323435e-03],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32)]


In [61]:
with open('xx.pkl', 'wb') as f:
    pickle.dump(sim.grid_v_in.grad.to_numpy(), f)

print(sim.grid_v_out.to_numpy()[36, 2, 31])
print(sim.ground_friction)

[0. 0. 0.]
5.0


In [30]:
print(sim.ground_height)
print(sim.dt)

3.0
1e-04


In [31]:
state0 = sim.get_state(19)
state1 = sim.get_state(19)

from utils import dump_taichi_grad

dump_taichi_grad(sim, 3)

{'state': [array([[0.4525263 , 0.14798051, 0.4914687 ],
         [0.513431  , 0.14476335, 0.45326093],
         [0.49218068, 0.05376873, 0.5370558 ],
         ...,
         [0.46270877, 0.09403352, 0.48051965],
         [0.4610474 , 0.14297779, 0.47313395],
         [0.45064017, 0.1402589 , 0.52029145]], dtype=float32),
  array([[ 9.3366265e-01, -8.5178202e-01, -9.3031281e-01],
         [ 0.0000000e+00, -8.9999996e-03,  0.0000000e+00],
         [ 1.1739137e-05, -8.9129871e-03, -2.5932659e-06],
         ...,
         [-5.7732854e-07, -9.0002790e-03,  6.3446763e-08],
         [ 0.0000000e+00, -8.9999977e-03,  0.0000000e+00],
         [ 1.3324900e+00, -1.1944015e+00, -1.3351561e+00]], dtype=float32),
  array([[[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 1.85243444e-12,  1.00000000e+00, -7.18497397e-12],
          [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],
  
         [[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [-4.07703038e-12,  1.000000

In [32]:
print(sim.p_vol)
print(sim.p_mass)
print(sim.mu)
print(sim.lam)
print(sim.yield_stress)

6.103515625e-05
[6.1035156e-05 6.1035156e-05 6.1035156e-05 ... 6.1035156e-05 6.1035156e-05
 6.1035156e-05]
[2083.3333 2083.3333 2083.3333 ... 2083.3333 2083.3333 2083.3333]
[1388.8889 1388.8889 1388.8889 ... 1388.8889 1388.8889 1388.8889]
[1000. 1000. 1000. ... 1000. 1000. 1000.]


In [48]:
sim.primitives[0].xyz_limit.to_numpy()

array([[-1., -1., -1.],
       [ 2.,  2.,  2.]], dtype=float32)